<a href="https://colab.research.google.com/github/DarKenW/customer-driven-data-science/blob/main/Customer_Analytics_Purchase_Analytics_Conversion_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audiobooks business converison prediction case

This project is a returning lead scoring/conversion prediction project.    
We need to predict which customers are most likely to **buy the audiobook again** and we should put more marketing resources on this group of customers. 

**We also will try to find most important metrics for conversion**.

Engagement column names:
- book length
- avg. book length (total book length / number of purchase)
- price overall
- price avg
- review
- review (10/10)
- minute listened
- completion
- support requests (from password forgotten to assistance on using the platform)
- last visited minus purchase date
- target

**Conversion** definition: input data represents 2 year worth of engagement and targets are 6 month of data to check conversion (buy another book or not).

## Preprocess the data. Balance the dataset. Standardize the data. 

Create 3 datasets: training, validation, and test. Save the newly created sets in a tensor friendly format (e.g. *.npz)

Since we are dealing with real life data, we will need to preprocess it a bit.

If you want to know how to do that, go through the code with comments. In any case, this should do the trick for most datasets organized in the way: many inputs, and then 1 cell containing the targets (supersized learning datasets). Keep in mind that a specific problem may require additional preprocessing.

Note that we have removed the header row, which contains the names of the categories. We simply need the numerical data.

### Extract the data from the csv

In [29]:
import numpy as np

# We will use the StandardScaler module, so we can later deploy the model
from sklearn.preprocessing import StandardScaler

import pickle

# df_purchase = pd.read_csv('purchase data.csv')

from google.colab import drive
drive.mount('/content/drive')

# Load the data, contained in the segmentation data csv file.
GD_PATH = '/content/drive/MyDrive/扬FAANG起航/单项准备/customer analytics/'
#df_purchase = pd.read_csv(GD_PATH+'Audiobooks_data.csv', index_col = 0)

# Load the data
raw_csv_data = np.loadtxt(GD_PATH+'Audiobooks_data.csv',delimiter=',')

# The inputs are all columns in the csv, except for the first one and the last one
# The first column is the arbitrary ID, while the last contains the targets

unscaled_inputs_all = raw_csv_data[:,1:-1]

# The targets are in the last column. That's how datasets are conventionally organized.
targets_all = raw_csv_data[:,-1]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
raw_csv_data

array([[8.7300e+02, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [6.1100e+02, 1.4040e+03, 2.8080e+03, ..., 0.0000e+00, 1.8200e+02,
        1.0000e+00],
       [7.0500e+02, 3.2400e+02, 3.2400e+02, ..., 1.0000e+00, 3.3400e+02,
        1.0000e+00],
       ...,
       [2.8671e+04, 1.0800e+03, 1.0800e+03, ..., 0.0000e+00, 2.9000e+01,
        0.0000e+00],
       [3.1134e+04, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.2832e+04, 1.6200e+03, 1.6200e+03, ..., 0.0000e+00, 9.0000e+01,
        0.0000e+00]])

In [31]:
raw_csv_data.shape

(14084, 12)

### Balance the dataset

We need to creat balanced dataset that has equal number of 0 and 1 in target column

In [32]:
# There are different Python packages that could be used for balancing
# Here we approach the problem manually, so you can observe the inner workings of the balancing process

# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets_all))

# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
# Declare a variable that will do that:
indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [33]:
# Crete a standar scaler object
scaler_deep_learning = StandardScaler()
# Fit and transform the original data
# Essentially, we calculate and STORE the mean and variance of the data in the scaler object
# At the same time we standrdize the data using this information
# Note that the mean and variance remain recorded in the scaler object
scaled_inputs = scaler_deep_learning.fit_transform(unscaled_inputs_equal_priors)

### Shuffle the data

In [34]:
# When the data was collected it was actually arranged by date
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
# Since we will be batching, we want the data to be as randomly spread out as possible
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [35]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too. Note that each time you rerun this code, 
# you will get different values, as each time they are shuffled randomly.
# Normally you preprocess ONCE, so you need not rerun this code once it is done.
# If you rerun this whole sheet, the npzs will be overwritten with your newly preprocessed data.

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1792.0 3579 0.5006985191394244
224.0 447 0.5011185682326622
221.0 448 0.49330357142857145


### Save the three datasets in *.npz

In [36]:
# Save the three datasets in *.npz.
# In the next lesson, you will see that it is extremely valuable to name them in such a coherent way!

np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

### Save the scaler

In [37]:
# Similar to how we have saved the scaler files before, we also save this scaler, so we can apply in on new data
pickle.dump(scaler_deep_learning, open('scaler_deep_learning.pickle', 'wb'))

## Problem

You are given data from an Audiobook app. Logically, it relates only to the audio versions of books. Each customer in the database has made a purchase at least once, that's why he/she is in the database. We want to create a machine learning algorithm based on our available data that can predict if a customer will buy again from the Audiobook company.

The main idea is that if a customer has a low probability of coming back, there is no reason to spend any money on advertizing to him/her. If we can focus our efforts ONLY on customers that are likely to convert again, we can make great savings. Moreover, this model can identify the most important metrics for a customer to come back again. Identifying new customers creates value and growth opportunities.

You have a .csv summarizing the data. There are several variables: Customer ID, Book length in mins_avg (average of all purchases), Book length in minutes_sum (sum of all purchases), Price Paid_avg (average of all purchases), Price paid_sum (sum of all purchases), Review (a Boolean variable), Review (out of 10), Total minutes listened, Completion (from 0 to 1), Support requests (number), and Last visited minus purchase date (in days).

So these are the inputs (excluding customer ID, as it is completely arbitrary. It's more like a name, than a number).

The targets are a Boolean variable (so 0, or 1). We are taking a period of 2 years in our inputs, and the next 6 months as targets. So, in fact, we are predicting if: based on the last 2 years of activity and engagement, a customer will convert in the next 6 months. 6 months sounds like a reasonable time. If they don't convert after 6 months, chances are they've gone to a competitor or didn't like the Audiobook way of digesting information. 

The task is simple: create a machine learning algorithm, which is able to predict if a customer will buy again. 

This is a classification problem with two classes: won't buy and will buy, represented by 0s and 1s. 

Good luck!

# Modeling Part

## Create the machine learning algorithm

### Import the relevant libraries

In [38]:
# we must import the libraries once again since we haven't imported them in this file
import numpy as np
import tensorflow as tf
import pandas as pd

### Data

In [39]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
#npz = np.load(GD_PATH+'Audiobooks_data_train.npz')
npz = np.load('Audiobooks_data_train.npz')
# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(np.float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(np.int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### Model
Outline, optimizers, loss, early stopping and training

In [40]:
# Optionally set the input size. We won't be using it, but in some cases it is beneficial
# input_size = 10
# Set the output size
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Epoch 1/100
36/36 - 1s - loss: 0.5023 - accuracy: 0.8030 - val_loss: 0.3784 - val_accuracy: 0.8814
Epoch 2/100
36/36 - 0s - loss: 0.3515 - accuracy: 0.8718 - val_loss: 0.3063 - val_accuracy: 0.8904
Epoch 3/100
36/36 - 0s - loss: 0.3199 - accuracy: 0.8793 - val_loss: 0.2853 - val_accuracy: 0.8949
Epoch 4/100
36/36 - 0s - loss: 0.3035 - accuracy: 0.8843 - val_loss: 0.2682 - val_accuracy: 0.9083
Epoch 5/100
36/36 - 0s - loss: 0.2891 - accuracy: 0.8868 - val_loss: 0.2563 - val_accuracy: 0.9195
Epoch 6/100
36/36 - 0s - loss: 0.2790 - accuracy: 0.8910 - val_loss: 0.2484 - val_accuracy: 0.9239
Epoch 7/100
36/36 - 0s - loss: 0.2726 - accuracy: 0.8949 - val_loss: 0.2488 - val_accuracy: 0.9195
Epoch 8/100
36/36 - 0s - loss: 0.2660 - accuracy: 0.8969 - val_loss: 0.2413 - val_accuracy: 0.9239
Epoch 9/100
36/36 - 0s - loss: 0.2614 - accuracy: 0.8977 - val_loss: 0.2465 - val_accuracy: 0.9195
Epoch 10/100
36/36 - 0s - loss: 0.2567 - accuracy: 0.9011 - val_loss: 0.2415 - val_accuracy: 0.9239


## Test the model

As we discussed in the lectures, after training on the training data and validating on the validation data, we test the final prediction power of our model by running it on the test dataset that the algorithm has NEVER seen before.

It is very important to realize that fiddling with the hyperparameters overfits the validation dataset. 

The test is the absolute final instance. You should not test before you are completely done with adjusting your model.

If you adjust your model after testing, you will start overfitting the test dataset, which will defeat its purpose.

In [41]:
# declare two variables that are going to contain the two outputs from the evaluate function
# they are the loss (which is there by default) and whatever was specified in the 'metrics' argument when fitting the model
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 1ms/step - loss: 0.2409 - accuracy: 0.9196


In [42]:
# Print the result in neatly formatted
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.24. Test accuracy: 91.96%


## Obtain the probability for a customer to convert

In [43]:
# We can predict the probability of each class using the 'predict' method
# The output comes in a scientific format
# Please uncomment the round() method to achive a rounded result (not scientific notation)
model.predict(test_inputs)#.round(2)

array([[2.31893107e-01, 7.68106878e-01],
       [6.47473708e-02, 9.35252607e-01],
       [9.59721744e-01, 4.02782559e-02],
       [1.19577512e-01, 8.80422473e-01],
       [9.78049815e-01, 2.19501350e-02],
       [1.27077103e-01, 8.72922838e-01],
       [8.66629899e-01, 1.33370161e-01],
       [1.36372223e-01, 8.63627732e-01],
       [2.91843973e-02, 9.70815599e-01],
       [1.35610346e-03, 9.98643935e-01],
       [2.03454107e-01, 7.96545863e-01],
       [3.18770222e-02, 9.68122900e-01],
       [4.20554839e-02, 9.57944512e-01],
       [2.21621349e-01, 7.78378665e-01],
       [2.23597899e-01, 7.76402056e-01],
       [8.72267783e-01, 1.27732188e-01],
       [3.28227013e-01, 6.71773016e-01],
       [3.32203950e-03, 9.96677995e-01],
       [9.73245144e-01, 2.67548636e-02],
       [1.28982723e-01, 8.71017277e-01],
       [3.50202143e-01, 6.49797857e-01],
       [8.14149797e-01, 1.85850203e-01],
       [1.04413601e-02, 9.89558578e-01],
       [7.81930536e-02, 9.21806931e-01],
       [3.008305

In [44]:
# Alternatively, we can get only the second column
# The main idea is that we are often interested in ONLY ONE of the two outcomes
# In this case we would like to know if the customer will convert again
# Once more, we can round to 0 digits, to achieve only 0s or 1s
model.predict(test_inputs)[:,1].round(0)

array([1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1.,
       0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
       0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
       0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0.,
       0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 1., 1.

In [45]:
# A much better approach here would be to use argmax (arguments of the maxima)
# Argmax indicates the position of the highest argument row-wise or column-wise
# In our case, we want ot know which COLUMN has the higher argument (probability), therefore we set axis=1 (for columns)
# The output would be the column ID with the highest argument for each observation (row)
# For instance, the first observation (in our output) was [0.93,0.07]
# np.argmax([0.93,0.07], axis=1) would find that 0.91 is the higher argument (higher probability) and return 0
# This method is great for multi-class problems as it is independent of the number of classes
np.argmax(model.predict(test_inputs),axis=1)

array([1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,

## Save the model

In [46]:
# Finally we save the model using the built-in method TensorFlow method
# We choose the name and the file extension
# Since the HDF format is optimal for large numerical objects, that's our preferred choice here (and the one recommended by TF)
# The proper extension is .h5 to indicate HDF, version 5
model.save('audiobooks_model.h5') 

# Predicting on new data

## Import the relevant libraries



In [47]:
# As usual we are starting a new notebook and we need to import all relevant packages
import numpy as np
import tensorflow as tf
import pickle

## Load the scaler and the model

In [48]:
# To load the scaler we use the pickle method load
scaler_deep_learning = pickle.load(open('scaler_deep_learning.pickle', 'rb'))
# To load the model, we use the TensorFlow (Keras) function relevant for the operation
model = tf.keras.models.load_model('audiobooks_model.h5')

# Note that since we did not specify the input shape of our inputs in the modeling part, we get a warning
# For feed-forward neural networks such as ours that is not an issue

## Load the new data

In [49]:
# The new data is located in 'New_Audiobooks_Data.csv'
# To keep everything as before, we must specify the delimiter explicitly
raw_data = np.loadtxt(GD_PATH+'New_Audiobooks_Data.csv',delimiter=',')
# We are interested in all data except for the first column (ID)
# Note that there are no targets in this CSV file (we don't know the behavior of these clients, yet!)
new_data_inputs = raw_data[:,1:]

## Predict the probability of a customer to convert

In [50]:
# Scale the new data in the same way we scaled the train data
new_data_inputs_scaled = scaler_deep_learning.transform(new_data_inputs)

In [51]:
# Predict the probability of each customer to convert
# Here we have also taken only the second column and rounded to 2 digits after the dot
model.predict(new_data_inputs_scaled)[:,1].round(2)

array([0.05, 0.  , 0.04, 1.  , 0.01, 0.03, 0.03, 0.13, 0.02, 0.7 , 0.  ,
       0.78, 0.95, 0.  , 0.12, 0.12, 0.87, 0.63, 0.84, 0.94, 1.  , 1.  ,
       0.97, 0.01, 0.  , 0.99, 0.39, 0.  , 1.  , 1.  ], dtype=float32)

In [52]:
# Implement the better approach which is independent of the number of classes
np.argmax(model.predict(new_data_inputs_scaled),1)

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1])